In [22]:
import os
import numpy as np
import torch
torch.set_num_threads(1)
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import argparse
from tqdm import tqdm
import logging
from model import TransformerVAE
from utils import AverageMeter
from dataset import get_dataloader
from model.losses import VAELoss, div_loss
from dataset import ReactionDataset


In [23]:
import torch

torch.multiprocessing.set_start_method('spawn')


In [24]:
# import dill as pickle
from dataset import ReactionDataset
from torch.utils.data import DataLoader
from model import TransformerVAE
import pdb
from tqdm import tqdm
# from model import HuBERTEncoder


print ('num threads: ', torch.get_num_threads())
# torch.set_num_threads(1)
audio_flag = True
model = TransformerVAE(
    img_size=256,    
    use_hubert=audio_flag,
audio_dim=128, # becomes irrelevant in case for hubert
max_seq_len=751, 
seq_len=750)

dataset = ReactionDataset('../data', 'data/sample_udiva.csv', clip_length=750, use_raw_audio=audio_flag, load_3dmm_l=True, mode='val')
shuffle = True 
# sample = dataset[0]
# hubert = HuBERTEncoder(1024)
# hubert = hubert.cuda()

train_loader = DataLoader(dataset=dataset, batch_size=2, shuffle=shuffle, num_workers=2)

checkpoints = torch.load('./results/marlin_hubert_27jun/cur_checkpoint.pth', map_location=torch.device('cpu'))
state_dict = checkpoints['state_dict']
model.load_state_dict(state_dict)
model = model.cuda()
# model.eval()
# speaker_video_clip, speaker_audio_clip, _, _, _, _, listener_emotion, listener_3dmm, listener_references = dataset[0]
# print (listener_3dmm.shape)
# print ("enumeratin")
for batch_idx, (speaker_video_clip, 
                speaker_video_orig, 
                speaker_audio_clip, speaker_emotion, _, 
                listener_video_clip, _, listener_emotion, listener_3dmm, listener_references) in enumerate(tqdm(train_loader)):

    print ('speaker video shape: ', speaker_video_clip.shape)
    print ('speaker audio shape: ', speaker_audio_clip.shape)
    print ('listener 3d mm shape: ', listener_3dmm.shape)
    print ("running inference...")
    
    with torch.no_grad():
        listener_3dmm_out, listener_emotion_out, distribution = model(speaker_video_clip.cuda(), speaker_audio_clip.cuda())

print ("done.")
# print ('Batch idx: ', batch_idx)
# train_loader = get_dataloader(args, "data/train.csv", load_ref=False, load_video_l=False)

num threads:  1
Using HuBERT.


  0%|                                                                                                                                                   | 0/1 [00:00<?, ?it/s]

speaker video shape:  torch.Size([1, 23, 1024])
speaker audio shape:  torch.Size([1, 480000])
listener 3d mm shape:  torch.Size([1, 750, 58])
running inference...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.95s/it]

done.


In [27]:
from model.losses import MSELoss

In [34]:
div_loss(listener_3dmm_out, listener_3dmm_out)

tensor(1., device='cuda:0')

In [32]:
listener_3dmm_out[0].shape

torch.Size([750, 58])

In [2]:
import moviepy.editor

In [1]:
import pandas as pd
df = pd.read_csv('../data/train_neg.csv')

In [6]:
df['speaker_path']

0          UDIVA/talk/023102/FC1/8
1          UDIVA/talk/023102/FC1/9
2         UDIVA/talk/023102/FC1/10
3         UDIVA/talk/023102/FC1/14
4         UDIVA/talk/023191/FC1/19
                   ...            
2629     UDIVA/animal/140170/FC1/7
2630    UDIVA/animal/140170/FC1/10
2631    UDIVA/animal/140170/FC1/11
2632    UDIVA/animal/140170/FC1/19
2633    UDIVA/animal/140170/FC1/20
Name: speaker_path, Length: 2634, dtype: object